In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams
from sentence_transformers import SentenceTransformer
import numpy as np
from pathlib import Path
import json
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()  # loads .env into environment variables

from huggingface_hub import login
login(os.getenv("HF_TOKEN"))
# ----------------------------
# Step 1: Connect to Qdrant
# ----------------------------
# Option 1: In-memory (no Docker)
client = QdrantClient(":memory:")

# # Option 2: Local Qdrant server
# client = QdrantClient("http://localhost:6333")

# ----------------------------
# Step 2: Initialize model
# ----------------------------
model = SentenceTransformer("intfloat/multilingual-e5-large")
model.save("models/multilingual-e5-large") #save model locally (do only once)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams
from sentence_transformers import SentenceTransformer
import numpy as np
from pathlib import Path
import json
import pandas as pd

model = SentenceTransformer("models/multilingual-e5-large")

client = QdrantClient(":memory:")
# client = QdrantClient(host="localhost", port=6333)



In [3]:


# from pathlib import Path
# import json
# import pandas as pd

# # ----------------------------
# # Step 3: Load first 100 articles from JSON
# # ----------------------------
# path = Path("../wikiextractor/articles_fr_withLinks.json")
# # ----------------------------
# # Step 3: Load articles from JSON (all lines into DataFrame)
# # ----------------------------
# # ... existing code ...
# ARTICLES_NUMBER = None  # None = load all; set an int to cap when sampling

# def load_jsonl_to_df(path, max_rows=None):
#     rows = []
#     with open(path, "r", encoding="utf-8") as f:
#         for i, line in enumerate(f):
#             if max_rows is not None and i >= max_rows:
#                 break
#             try:
#                 rows.append(json.loads(line))
#             except json.JSONDecodeError:
#                 continue
#     df = pd.DataFrame(rows)
#     # Ensure consistent types and fill missing fields
#     if "id" in df.columns:
#         df["id"] = pd.to_numeric(df["id"], errors="coerce").astype("Int64").astype(int)
#     df["title"] = df.get("title", "").fillna("")
#     df["url"] = df.get("url", "").fillna("")
#     df["text"] = df.get("text", "").fillna("")
#     return df

# df = load_jsonl_to_df(path, max_rows=ARTICLES_NUMBER)
# print(f"✅ Loaded {len(df)} articles into a single DataFrame")
# # ... existing code ...

# # ----------------------------
# # Step 3: Create collection
# # ----------------------------
# vector_size = model.get_sentence_embedding_dimension()

# client.recreate_collection(
#     collection_name="wikipedia_fr",
#     vectors_config=VectorParams(size=vector_size, distance="Cosine"),
# )

# ----------------------------
# Step 3: Load articles from JSON (Polars)
# ----------------------------
from pathlib import Path
import json
import polars as pl

path = Path("../wikiextractor/articles_fr_withLinks.json")

ARTICLES_NUMBER = None  # None = load all; set an int to cap when sampling

def load_jsonl_to_df(path: Path, max_rows: int | None = None) -> pl.DataFrame:
    # If loading the full dataset, use Polars' fast NDJSON reader
    if max_rows is None:
        try:
            df = pl.read_ndjson(str(path))
        except Exception:
            # Fallback if the NDJSON reader hits malformed lines: manual parse
            rows = []
            with open(path, "r", encoding="utf-8") as f:
                for line in f:
                    try:
                        rows.append(json.loads(line))
                    except json.JSONDecodeError:
                        continue
            df = pl.DataFrame(rows)
    else:
        # Limited load: manual parse up to max_rows
        rows = []
        with open(path, "r", encoding="utf-8") as f:
            for i, line in enumerate(f):
                if i >= max_rows:
                    break
                try:
                    rows.append(json.loads(line))
                except json.JSONDecodeError:
                    continue
        df = pl.DataFrame(rows)

    # Ensure columns exist and normalize types/nulls
    cols = set(df.columns)

    if "id" in cols:
        # Cast id to Int64
        df = df.with_columns(pl.col("id").cast(pl.Int64))
    else:
        df = df.with_columns(pl.lit(None).cast(pl.Int64).alias("id"))

    if "title" in cols:
        df = df.with_columns(pl.col("title").cast(pl.Utf8).fill_null(""))
    else:
        df = df.with_columns(pl.lit("").alias("title"))

    if "url" in cols:
        df = df.with_columns(pl.col("url").cast(pl.Utf8).fill_null(""))
    else:
        df = df.with_columns(pl.lit("").alias("url"))

    if "text" in cols:
        df = df.with_columns(pl.col("text").cast(pl.Utf8).fill_null(""))
    else:
        df = df.with_columns(pl.lit("").alias("text"))

    return df

df = load_jsonl_to_df(path, max_rows=ARTICLES_NUMBER)
print(f"✅ Loaded {len(df)} articles into a single Polars DataFrame")


✅ Loaded 4498441 articles into a single Polars DataFrame


In [4]:
df.write_parquet("articles_fr_withLinks.parquet")

In [5]:
# ... existing code ...
vector_size = model.get_sentence_embedding_dimension()

client.recreate_collection(
    collection_name="wikipedia_fr",
    vectors_config=VectorParams(size=vector_size, distance="Cosine"),
)
# ... existing code ...

/var/folders/6x/x7v4d0vs261d025y0qz66bdh0000gn/T/ipykernel_1569/534072922.py:4: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [ ]:
# ----------------------------
# Step 4: Ingest articles into Qdrant with payload (Robust & Parallelized)
# ----------------------------
import polars as pl
import numpy as np
from qdrant_client.models import PointStruct
import gc
from tqdm.auto import tqdm

# --- Configuration ---
# Set the number of articles to process. Set to None to process all articles.
ARTICLES_TO_PROCESS = 50000 # For example, process the first 10,000 articles

chunk_size_words = 256
# Batch size for upserting points to Qdrant
points_batch_size = 512 

# --- Get model embedding dimension ---
embedding_dim = model.get_sentence_embedding_dimension()

# --- Core vectorization function (to be run in parallel) ---
def get_article_vector(text: str) -> np.ndarray:
    """
    Chunks text, encodes chunks, and returns the mean-pooled vector.
    This function is designed to be applied to each row by Polars' map_elements.
    """
    # 1. Chunk the text
    words = text.split()
    chunks = [" ".join(words[i:i+chunk_size_words]) for i in range(0, len(words), chunk_size_words)]
    
    # Handle empty texts by encoding a single empty string
    if not chunks:
        chunks = [""]
        
    # 2. Add the E5 model's required prefix
    prefixed_chunks = [f"passage: {chunk}" for chunk in chunks]
    
    # 3. Encode the chunks for this single article
    vectors = model.encode(prefixed_chunks, normalize_embeddings=True) # returns float32
    
    # 4. Mean-pool the vectors and ensure correct dtype
    if vectors.shape[0] > 0:
        return np.mean(vectors, axis=0).astype(np.float32)
    else:
        return np.zeros(embedding_dim, dtype=np.float32)

# --- Main processing ---
# Slice the DataFrame to process only the specified number of articles
if ARTICLES_TO_PROCESS is not None:
    df_to_process = df.slice(0, ARTICLES_TO_PROCESS)
else:
    df_to_process = df

print(f"🚀 Starting parallel vectorization of {df_to_process.height} articles...")

# Use `map_elements` to apply our function to each text in parallel.
df_with_vectors = df_to_process.with_columns(
    vector=pl.col("text").map_elements(
        get_article_vector,
        # Ensure the return type matches the function's output exactly
        return_dtype=pl.Array(pl.Float32, embedding_dim),
        strategy="thread_local" # Use multiple threads for parallel execution
    )
)

print("✅ Vectorization complete. Now creating and upserting points...")

# --- Create and upsert points in batches (Optimized) ---
points = []
total_rows = df_with_vectors.height

# Extract columns into Python lists for fast iteration
ids = df_with_vectors.get_column("id").to_list()
vectors_list = df_with_vectors.get_column("vector").to_list()
payloads = df_with_vectors.select(["id", "title", "url"]).to_dicts()

for i in tqdm(range(total_rows), desc="Upserting to Qdrant"):
    points.append(
        PointStruct(
            id=int(ids[i]),
            vector=vectors_list[i],
            payload=payloads[i]
        )
    )
    
    # Upsert in batches to Qdrant for network efficiency
    if len(points) >= points_batch_size:
        client.upsert(collection_name="wikipedia_fr", points=points, wait=False)
        points.clear()

# --- Final flush for any remaining points ---
if points:
    client.upsert(collection_name="wikipedia_fr", points=points, wait=True)
    points.clear()
    

# Clean up memory
gc.collect()

print(f"✅ Ingestion complete. Upserted embeddings for {total_rows} articles.")

🚀 Starting parallel vectorization of 50000 articles...


In [ ]:
# ----------------------------
# Step 6: Query similar articles
# ----------------------------
query_title = "Math"

query_row = df[df["title"].str.lower() == query_title.lower()]
if not query_row.empty:
    query_text = query_row.iloc[0]["text"]
else:
    query_text = query_title

query_vector = model.encode([query_text], normalize_embeddings=True)[0]

results = client.search(
    collection_name="wikipedia_fr",
    query_vector=query_vector,
    limit=5
)

print(f"\n🔍 Top matches for '{query_title}':")
for r in results:
    print(f"• {r.payload['title']} (score={r.score:.3f})")


🔍 Top matches for 'Math':
• Algèbre de Boole (logique) (score=0.804)
• Algèbre générale (score=0.787)
• Algorithmique (score=0.785)
• Algèbre linéaire (score=0.785)
• Algorithme (score=0.781)


/tmp/ipykernel_18505/433745059.py:14: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [ ]:
# ----------------------------
# Step 6: Query similar articles
# ----------------------------
# ... existing code ...
import numpy as np

def build_query_vector_from_article_text(text, model, chunk_size_words=256):
    words = text.split()
    chunks = [" ".join(words[i:i+chunk_size_words]) for i in range(0, len(words), chunk_size_words)]
    prefixed = [f"query: {chunk}" for chunk in chunks]
    vecs = model.encode(prefixed, normalize_embeddings=True)
    return np.mean(vecs, axis=0)

query_title = "Math"
query_row = df[df["title"].str.lower() == query_title.lower()]

if not query_row.empty:
    # Use whole article text (chunked) for the query vector
    query_text = query_row.iloc[0]["text"]
    query_vector = build_query_vector_from_article_text(query_text, model)
else:
    # Fallback: use the title as plain query
    query_vector = model.encode([f"query: {query_title}"], normalize_embeddings=True)[0]

results = client.search(
    collection_name="wikipedia_fr",
    query_vector=query_vector,
    limit=5
)

print(f"\n🔍 Top matches for '{query_title}':")
for r in results:
    print(f"• {r.payload['title']} (score={r.score:.3f})")
# ... existing code ...


🔍 Top matches for 'Math':
• Algèbre de Boole (logique) (score=0.818)
• Algèbre linéaire (score=0.804)
• Abréviations en informatique H (score=0.804)
• Aichi (score=0.804)
• André Marie Ampère (score=0.804)


/tmp/ipykernel_18505/1095570288.py:25: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [ ]:
# ----------------------------
# Step 6: Query similar articles
# ----------------------------
# ... existing code ...
from qdrant_client.http import models as rest

query_title = "Math"
query_row = df[df["title"].str.lower() == query_title.lower()]

if not query_row.empty:
    # Assume point id in Qdrant equals the article 'id' stored in df
    article_id = int(query_row.iloc[0]["id"])
    results = client.recommend(
        collection_name="wikipedia_fr",
        positive=[article_id],
        limit=5
    )
else:
    # Fallback to text-based search using query prefix
    query_vector = model.encode([f"query: {query_title}"], normalize_embeddings=True)[0]
    results = client.search(
        collection_name="wikipedia_fr",
        query_vector=query_vector,
        limit=5
    )

print(f"\n🔍 Top matches for '{query_title}':")
for r in results:
    print(f"• {r.payload['title']} (score={r.score:.3f})")
# ... existing code ...


🔍 Top matches for 'Math':
• Algèbre de Boole (logique) (score=0.818)
• Algèbre linéaire (score=0.804)
• Abréviations en informatique H (score=0.804)
• Aichi (score=0.804)
• André Marie Ampère (score=0.804)


/tmp/ipykernel_18505/340894359.py:21: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [ ]:
# ----------------------------
# Step 6: Article-to-article similarity by full content
# ----------------------------
# ... existing code ...
import numpy as np
from qdrant_client.http import models as rest

def encode_article_text(text, model, chunk_size_words=256):
    """Encode full article by chunking, using E5 'passage:' prefix and mean-pooling."""
    words = text.split()
    chunks = [" ".join(words[i:i+chunk_size_words]) for i in range(0, len(words), chunk_size_words)]
    prefixed = [f"passage: {chunk}" for chunk in chunks]
    vecs = model.encode(prefixed, normalize_embeddings=True)
    return np.mean(vecs, axis=0)

def find_point_id_by_payload_id(client, collection_name, article_id):
    """Locate the stored point by payload 'id'."""
    points, next_page = client.scroll(
        collection_name=collection_name,
        scroll_filter=rest.Filter(
            must=[rest.FieldCondition(key="id", match=rest.MatchValue(value=str(article_id)))]
        ),
        limit=1,
        with_payload=True,
        with_vectors=False
    )
    if points:
        return points[0].id
    # Also try numeric payload ids, if stored as int
    points, next_page = client.scroll(
        collection_name=collection_name,
        scroll_filter=rest.Filter(
            must=[rest.FieldCondition(key="id", match=rest.MatchValue(value=int(article_id)))]
        ),
        limit=1,
        with_payload=True,
        with_vectors=False
    )
    return points[0].id if points else None

def get_similar_articles_by_id(client, collection_name, article_id, k=10):
    """Nearest neighbors via Qdrant recommend, using the stored vector."""
    point_id = find_point_id_by_payload_id(client, collection_name, article_id)
    if point_id is None:
        return None
    return client.recommend(collection_name=collection_name, positive=[point_id], limit=k)

def get_similar_articles_by_text(client, collection_name, text, model, k=10):
    """Fallback: nearest neighbors by encoding full article text."""
    vec = encode_article_text(text, model)
    return client.search(collection_name=collection_name, query_vector=vec, limit=k)

def get_title_by_id(article_id: int) -> str:
    # Try DataFrame first
    row = df[df["id"].astype(str) == str(article_id)]
    if not row.empty and pd.notna(row.iloc[0]["title"]):
        return str(row.iloc[0]["title"])
    # Fallback: try Qdrant payload
    try:
        pt = client.retrieve(
            collection_name=collection_name,
            ids=[int(article_id)],
            with_payload=True,
            with_vectors=False
        )
        if pt and pt[0].payload and pt[0].payload.get("title"):
            return str(pt[0].payload["title"])
    except Exception:
        pass
    return "(unknown title)"

collection_name = "wikipedia_fr"

# Example: article-to-article neighbors for two articles
for query_article_id in [7, 9]:
    query_title = get_title_by_id(int(query_article_id))
    print(f"\n🔎 Similar articles for: {query_title} (id={query_article_id})")
    results = get_similar_articles_by_id(client, collection_name, query_article_id, k=10)

    if results is None:
        # Fallback to local encoding if the article vector is not stored in Qdrant
        row = df[df["id"].astype(str) == str(query_article_id)]
        if row.empty:
            print(f"  • Article id={query_article_id} not found in DataFrame.")
            continue
        text = row.iloc[0]["text"]
        results = get_similar_articles_by_text(client, collection_name, text, model, k=10)

    for r in results:
        print(f"  • {r.payload.get('title')} (id={r.payload.get('id')}, score={r.score:.3f})")
# ... existing code ...


🔎 Similar articles for: Algèbre linéaire (id=7)
  • Algèbre générale (id=9, score=0.962)
  • Algorithmique (id=10, score=0.958)
  • Astronomie (id=64, score=0.953)
  • Algèbre de Boole (logique) (id=24, score=0.950)
  • Algorithme (id=19, score=0.945)
  • Antiquité (id=123, score=0.936)
  • Assistant personnel (id=124, score=0.936)
  • Anthropologie (id=152, score=0.935)
  • Antoine de Saint-Exupéry (id=97, score=0.928)
  • Asie (id=159, score=0.928)

🔎 Similar articles for: Algèbre générale (id=9)
  • Algèbre linéaire (id=7, score=0.962)
  • Algorithmique (id=10, score=0.946)
  • Astronomie (id=64, score=0.936)
  • Algorithme (id=19, score=0.932)
  • Algèbre de Boole (logique) (id=24, score=0.931)
  • Anthropologie (id=152, score=0.919)
  • Antiquité (id=123, score=0.919)
  • Assistant personnel (id=124, score=0.917)
  • Asie (id=159, score=0.908)
  • Apple (id=63, score=0.908)


/tmp/ipykernel_18505/2584421462.py:46: DeprecationWarning: `recommend` method is deprecated and will be removed in the future. Use `query_points` instead.
  return client.recommend(collection_name=collection_name, positive=[point_id], limit=k)
/tmp/ipykernel_18505/2584421462.py:46: DeprecationWarning: `recommend` method is deprecated and will be removed in the future. Use `query_points` instead.
  return client.recommend(collection_name=collection_name, positive=[point_id], limit=k)


In [ ]:
# ----------------------------
# Step 6: Export recomputed similarity links (top-k neighbors)
# ----------------------------
# ... existing code ...
import csv

def iter_all_point_ids(client, collection_name, batch_size=1024):
    """Yield all point IDs in the collection (without vectors)."""
    next_page = None
    while True:
        points, next_page = client.scroll(
            collection_name=collection_name,
            limit=batch_size,
            with_payload=True,
            with_vectors=False,
            offset=next_page
        )
        if not points:
            break
        for p in points:
            # Prefer payload 'id' if present, else use point id
            payload_id = p.payload.get("id")
            yield payload_id if payload_id is not None else p.id
        if next_page is None:
            break

def export_similarity_links(client, collection_name, k, out_csv_path):
    """For each article, record top-k similar neighbors to CSV."""
    with open(out_csv_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["source_id", "target_id", "score"])
        count = 0
        for source_id in iter_all_point_ids(client, collection_name):
            # Resolve to actual point ID if payload_id is used
            neighbors = get_similar_articles_by_id(client, collection_name, source_id, k=k)
            if neighbors is None:
                # Skip if the source article is missing as a stored vector
                continue
            for r in neighbors:
                target_id = r.payload.get("id", r.id)
                if str(target_id) == str(source_id):
                    continue
                writer.writerow([source_id, target_id, f"{r.score:.6f}"])
            count += 1
            if count % 1000 == 0:
                print(f"Processed {count} articles...")
    print(f"✅ Exported recomputed links to {out_csv_path}")

# Generate a 10-NN hyperlink graph (adjust k as needed)
export_similarity_links(client, collection_name="wikipedia_fr", k=10, out_csv_path="recomputed_similarity_links.csv")
# ... existing code ...

/tmp/ipykernel_18505/2584421462.py:46: DeprecationWarning: `recommend` method is deprecated and will be removed in the future. Use `query_points` instead.
  return client.recommend(collection_name=collection_name, positive=[point_id], limit=k)


✅ Exported recomputed links to recomputed_similarity_links.csv
